In [9]:
%pip install ultralytics


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install kagglehub


Note: you may need to restart the kernel to use updated packages.


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("fatihkgg/affectnet-yolo-format")

print("Path to dataset files:", path)

100%|██████████| 258M/258M [00:11<00:00, 22.9MB/s] 

Extracting files...


Path to dataset files: /home/sagemaker-user/.cache/kagglehub/datasets/fatihkgg/affectnet-yolo-format/versions/2


In [6]:
%cp -r /home/sagemaker-user/.cache/kagglehub/datasets/fatihkgg/affectnet-yolo-format/versions/2/YOLO_format .

In [6]:
import os
import yaml
from ultralytics import YOLO
import numpy as np
from pathlib import Path
import torch.nn as nn

In [7]:
# ---------------- Hyper‑parameters -----------------
YAML_FILE = './YOLO_format/data.yaml'
EPOCHS = 10000
BATCH  = 32
IMGSZ  = 640
WORKERS = 8
PATIENCE = 50
SAVE_PERIOD = 50
DEVICE = '0' 
RESUME = False
# ---------------------------------------------------

# Training

In [8]:
model = YOLO('yolov8n.pt') 

In [ ]:
results = model.train(
    data=YAML_FILE,
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device=DEVICE,
    workers=WORKERS,
    project='runs/train',
    name='affectnet_yolov8',
    patience=PATIENCE,
    save=True,
    save_period=SAVE_PERIOD,
    plots=True,
    resume=RESUME
)

# Evaluation

In [2]:
def load_data_yaml(path):
    """Load and return YAML content."""
    with open(path, 'r') as f:
        data = yaml.safe_load(f)
    return data

In [4]:
model = YOLO('/home/sagemaker-user/runs/train/affectnet_yolov84/weights/best.pt')
data = load_data_yaml(YAML_FILE)

In [5]:
metrics = model.val(
    split='train',        # evaluate only on test
    data=YAML_FILE,
    imgsz=IMGSZ,
    batch=BATCH,
    device=DEVICE,
    save=True,
    workers=WORKERS,
    resume=False,                      # set True if picking up from last.pt
    plots=True,
    project="runs",      
    name="train"
)

Ultralytics 8.3.138 🚀 Python-3.12.9 torch-2.5.1 CUDA:0 (Tesla T4, 14918MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 1.4±0.3 ms, read: 12.6±8.7 MB/s, size: 8.8 KB)


val: Scanning /home/sagemaker-user/YOLO_format/train/labels.cache... 17101 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17101/17101 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 535/535 [01:53<00:00,  4.71it/s]


                   all      17101      17101      0.904      0.904      0.965      0.965
                 Anger       2339       2339      0.926      0.899      0.972      0.972
              Contempt       1996       1996      0.839      0.937       0.96       0.96
               Disgust       2242       2242      0.933      0.878      0.967      0.967
                  Fear       2021       2021      0.954       0.87      0.974      0.974
                 Happy       2154       2154      0.962      0.953      0.988      0.988
               Neutral       1616       1616      0.814      0.866      0.922      0.922
                   Sad       1914       1914      0.889      0.907      0.965      0.965
              Surprise       2819       2819      0.912      0.925      0.975      0.975
Speed: 0.3ms preprocess, 3.6ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/train3
